(〃￣︶￣)人(〃￣︶￣〃)人(〃￣︶￣〃)人(￣︶￣〃)

## Questions

1. For the baseline, should we keep the POS tag of the words as keys? i.e., 'physical.a' vs. 'physical'. 
   - And in the forward phase of the WSD1?

~~2. Creating WSD1, what should we use for the out features? ~~

   ~~- self.classifier = nn.Linear(o_dim, num_senses) or self.classifier = nn.Linear(o_dim, num_words)?~~

3. For the output of the first model, should batch.position be just of size 1? Or a tensor for the indexing?
   - word = hidden_states[range(hidden_states.shape[0]), batch.position]


4. AttributeError: 'BertTokenizer' object has no attribute 'batch_encode_plus' -> we should update transformers. But can we do that from the server?
   

# Word Sense Disambiguation using Neural Networks
Adam Ek

----------------------------------

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the instructions on [how to work on group assignments](https://github.com/sdobnik/computational-semantics/blob/master/README.md).

Write all your answers and the code in the appropriate boxes below.

----------------------------------

A problem with static distributional vectors is the difficulty of distinguishing between different *word senses*. We will continue our exploration of word vectors by considering *trainable vectors* or *word embeddings* for Word Sense Disambiguation (WSD).

The goal of word sense disambiguation is to train a model to find the sense of a word (homonyms of a word-form). For example, the word "bank" can mean "sloping land" or "financial institution". 

(a) "I deposited my money in the **bank**" (financial institution)

(b) "I swam from the river **bank**" (sloping land)

In case a) and b) we can determine that the meaning of "bank" based on the *context*. To utilize context in a semantic model we use *contextualized word representations*. Previously we worked with *static word representations*, i.e. the representation does not depend on the context. To illustrate we can consider sentences (a) and (b), the word **bank** would have the same static representation in both sentences, which means that it becomes difficult for us to predict its sense. What we want is to create representations that depend on the context, i.e. *contextualized embeddings*. 

We will create contextualized embeddings with Recurrent Neural Networks. You can read more about recurrent neural netoworks [here](https://colah.github.io/posts/2015-08-Understanding-LSTMs/). Your overall task in this lab is to create a neural network model that can disambiguate the word sense of 15 different words. 

In [41]:
# first we import some packages that we need
import torch
import torch.nn as nn
import torchtext
import torch.optim as optim
from torchtext.data import Field, BucketIterator, Iterator, TabularDataset
import numpy as np
# our hyperparameters (add more when/if you need them)
device = torch.device('cuda:2')

batch_size = 4
learning_rate = 0.001
epochs = 3
max_grad_norm = 5

# 1. Working with data

A central part of any machine learning system is the data we're working with. In this section we will split the data (the dataset is located here: ``wsd-data/wsd_data.txt``) into a training set and a test set. We will also create a baseline to compare our model against. Finally, we will use TorchText to transform our data (raw text) into a convenient format that our neural network can work with.

## Data

The dataset we will use contain different word sense for 15 different words. The data is organized as follows (values separated by tabs): 
- Column 1: word-sense
- Column 2: word-form
- Column 3: index of word
- Column 4: white-space tokenized context

### Splitting the data

Your first task is to seperate the data into a *training set* and a *test set*. The training set should contain 80% of the examples and the test set the remaining 20%. The examples for the test/training set should be selected **randomly**. Save each dataset into a .csv file for loading later. **[2 marks]**

In [2]:
from sklearn.model_selection import train_test_split

path = 'wsd-data/wsd_data.txt'

def data_split(path_to_dataset):
    
    with open(path_to_dataset) as d:
        di = d.readlines()
    
        train, test = train_test_split(di, test_size=0.20, random_state=0)

        with open('train.csv', 'w') as g:
            for l in train:
                g.write(l)
        with open('test.csv', 'w') as h:
            for l in test:
                h.write(l)

data_split(path)

### Creating a baseline

Your second task is to create a *baseline* for the task. A baseline is a "reality check" for a model, given a very simple heuristic/algorithmic/model solution to the problem, can our neural network perform better than this?
The baseline you are to create is the "most common sense" (MCS) baseline. For each word form, find the most commonly assigned sense to the word, and label a words with that sense. **[2 marks]**

E.g. In a fictional dataset, "bank" have two senses, "financial institution" which occur 5 times and "side of river" 3 times. Thus, all 8 occurences of bank is labeled "financial institution" and this yields an MCS accuracy of 5/8 = 62.5%. If a model obtain a higher score than this, we can conclude that the model *at least* is better than selecting the most frequent word sense.

In [3]:
def mcs_baseline(data):
    result = {}
    with open(data) as file:
        for line in file.readlines():
            split_line = line.split("\t")
            if split_line[1] in result:
                if split_line[0] in result[split_line[1]]:
                    result[split_line[1]][split_line[0]] += 1
                else:
                    result[split_line[1]][split_line[0]] = 1
            else:
                result[split_line[1]] = {split_line[0]: 1}
    
    # select most common sense for words
    for key, value in result.items():
        result[key] = max(value, key=value.get)
        # print(key, len(value.values()))
        
    return result

baseline = mcs_baseline("train.csv") 

### Creating data iterators

To train a neural network, we first need to prepare the data. This involves converting words (and labels) to a number, and organizing the data into batches. We also want the ability to shuffle the examples such that they appear in a random order.  

To do all of this we will use the torchtext library (https://torchtext.readthedocs.io/en/latest/index.html). In addition to converting our data into numerical form and creating batches, it will generate a word and label vocabulary, and data iterators than can sort and shuffle the examples. 

Your task is to create a dataloader for the training and test set you created previously. So, how do we go about doing this?

1) First we create a ``Field`` for each of our columns. A field is a function which tokenize the input, keep a dictionary of word-to-numbers, and fix paddings. So, we need four fields, one for the word-sense, one for the position, one for the lemma and one for the context. 

2) After we have our fields, we need to process the data. For this we use the ``TabularDataset`` class. We pass the name and path of the training and test files we created previously, then we assign which field to use in each column. The result is that each column will be processed by the field indicated. So, the context column will be tokenized and processed by the context field and so on. 

3) After we have processed the dataset we need to build the vocabulary, for this we call the function ``build_vocab()`` on the different ``Fields`` with the output from ``TabularDataset`` as input. This looks at our dataset and creates the necessary vocabularies (word-to-number mappings). 

4) Finally, the last step. In the last step we load the data objects given by the ``TabularDataset`` and pass it to the ``BucketIterator`` class. This class will organize our examples into batches and shuffle them around (such that for each epoch the model observe the examples in a different order). When we are done with this we can let our function return the data iterators and vocabularies, then we are ready to train and test our model!

Implement the dataloader. [**2 marks**]

*hint: for TabularDataset and BucketIterator use the class function splits()* 

In [57]:
def dataloader(path):
    
    whitespacer = lambda x: x.split(' ')

    # "fields" that process the different columns in our CSV files
    WORDSENSE = Field(tokenize    = whitespacer,
               lower       = True,
               batch_first = True)

    LEMMA = Field(tokenize    = whitespacer,
                  lower       = True,
                  batch_first = True)
    
    POSITION = Field(tokenize    = whitespacer,
                     sequential = False,
                     use_vocab = False, # To make sure you don't use the vocabulary for this field
                     batch_first = True)
    
    CONTEXT = Field(tokenize    = whitespacer,
                    lower       = True,
                    batch_first = True)
    
    # read the csv files
    train, test = TabularDataset.splits(path   = path,
                                        train  = 'train.csv',
                                        test   = 'test.csv',
                                        format = 'csv',
                                        fields = [('sense', WORDSENSE),
                                                  ('lemma', LEMMA),
                                                 ('position', POSITION),
                                                 ('context', CONTEXT)],
                                        skip_header       = True,
                                        csv_reader_params = {'delimiter':'\t',
                                                             'quotechar':'½'})
    
    # build vocabularies based on what our csv files contained and create word2id mapping
    WORDSENSE.build_vocab(train) #, min_freq=3) 
    LEMMA.build_vocab(train)
    CONTEXT.build_vocab(train)

    # create batches from our data, and shuffle them for each epoch
    train_iter, test_iter = BucketIterator.splits((train, test),
                                                  batch_size        = batch_size,
                                                  sort_within_batch = True,
                                                  sort_key          = lambda x: len(x.lemma), # context?
                                                  shuffle           = True,
                                                  device            = device)

    return train_iter, test_iter, WORDSENSE, LEMMA, POSITION, CONTEXT

# 2.1 Creating and running a Neural Network for WSD

In this section we will create and run a neural network to predict word senses based on *contextualized representations*.

### Model

We will use a bidirectional Long-Short-Term Memory (LSTM) network to create a representation for the sentences and a Linear classifier to predict the sense of each word.

When we initialize the model, we need a few things:

    1) An embedding layer: a dictionary from which we can obtain word embeddings
    2) A LSTM-module to obtain contextual representations
    3) A classifier that compute scores for each word-sense given *some* input


The general procedure is the following:

    1) For each word in the sentence, obtain word embeddings
    2) Run the embedded sentences through the RNN
    3) Select the appropriate hidden state
    4) Predict the word-sense 

**Suggestion for efficiency:**  *Use a low dimensionality (32) for word embeddings and the LSTM when developing and testing the code, then scale up when running the full training/tests*
    
Your tasks will be to create two different models (both follow the two outlines described above), described below:

In the first approach to WSD, you are to select the index of our target word (column 3 in the dataset) and predict the word sense. **[5 marks]**


Adam: "So in the first approach we want to use the LSTM (contextual) representation of the ambiguous word to predict its sense, so we need to **extract that representation and pass it to our prediction layer**"

In [58]:
class WSDModel_approach1(nn.Module):
    def __init__(self, num_words, num_senses, i_dim, o_dim):
        super(WSDModel_approach1, self).__init__()
        self.embeddings = nn.Embedding(num_words, i_dim) # Embeddings should capture the senses right?
        self.rnn = nn.LSTM(i_dim, o_dim, bidirectional=True, batch_first=True) # why not nn.RNN()??
        self.classifier = nn.Linear(o_dim*2, num_senses) # the output of the lstm (TIMES TWO) is the input of the linnear
    
    def forward(self, batch):
        embedded_batch = self.embeddings(batch.context)
        hidden_states, (final_hidden, cell_state) = self.rnn(embedded_batch) # you put the embedding in the context
        # hidden_states => multidim matrix of size [batch_size, sequence_lentght, o_dim]
        # select by index from hidden_states
        word = hidden_states[range(hidden_states.shape[0]), batch.position]
        
        return self.classifier(word)

In the second approach to WSD, you are to predict the word sense based on the final hidden state given by the RNN. **[5 marks]**

In [59]:
class WSDModel_approach2(nn.Module):
    def __init__(self, num_words, num_senses, i_dim, o_dim):
        super(WSDModel_approach2, self).__init__()
        self.embeddings = nn.Embedding(num_words, i_dim)
        self.rnn = nn.LSTM(i_dim, o_dim, bidirectional=True, batch_first=True)
        self.classifier = nn.Linear(o_dim*2, num_senses)
    
    def forward(self, batch):
        embedded_batch = self.embeddings(batch.context) 
        hidden_states, (final_hidden, cell_state) = self.rnn(embedded_batch)
        
        not_cool_approach = torch.cat((final_hidden[0], final_hidden[1]), dim=1)
        
        # cool_approach = torch.cat((final_hidden[0], cell_state[0], final_hidden[1], cell_state[1]), dim=1) # TO TRY L8R M8
        # final_hidden = [[...], [...]]
        # cell_state = [[...], [...]]
        
        # concatenate forward and backward OR concatenate final_hidden with cell_state
        pred = self.classifier(not_cool_approach)
        #fh = pred[range(pred.shape[1]), torch.tensor([0])] #<-- error here: index 2 is out of bounds for dimension 0 with size 2
        return pred # a "summary" of the whole sentence (just taking notes)

### Training and testing the model

Now we are ready to train and test our model. What we need now is a loss function, an optimizer, and our data. 

- First, create the loss function and the optimizer.
- Next, we iterate over the number of epochs (i.e. how many times we let the model see our data). 
- For each epoch, iterate over the dataset (``train_iter``) to obtain batches. Use the batch as input to the model, and let the model output scores for the different word senses.
- For each model output, calculate the loss (and print the loss) on the output and update the model parameters.
- Reset the gradients and repeat.
- After all epochs are done, test your trained model on the test set (``test_iter``) and calculate the total and per-word-form accuracy of your model.

Implement the training and testing of the model **[4 marks]**

**Suggestion for efficiency:** *when developing your model, try training and testing the model on one or two batches (for each epoch) of data to make sure everything works! It's very annoying if you train for N epochs to find out that something went wrong when testing the model, or to find that something goes wrong when moving from epoch 0 to epoch 1.*

In [60]:
def train(model, train_iter):

    loss = nn.CrossEntropyLoss(reduction='mean')
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model.train()

    for e in range(epochs):
        epoch_loss = 0


        for i, batch in enumerate(train_iter): 
            sentences = batch.context
            senses    = batch.sense

            # run sentences through the model
            output = model(batch) 

            # compute loss
            # output: from (B, L, C) to (B*L, C)
            # labels: from (B, C) to (B*C)
            # where B = batch size, L = sequence length and C the number of labels

            batch_loss  = loss(output, senses.view(-1))
            epoch_loss += batch_loss.item()

            # report results
            print(e, (i+1)*sentences.size(0), np.round(epoch_loss/(i+1),4),
                  end='\r')

            # calculate gradients
            batch_loss.backward()
            # update model weights
            optimizer.step()
            # reset gradients
            optimizer.zero_grad()
        print()
    return model

def test_model(model, test_iter):
    
    loss = nn.CrossEntropyLoss(reduction='mean')
    
    model.eval()
    # test model after all epochs are completed
    test_loss = 0

    # iterate over the test data and compute the class probabilities, same
    # procedure as before, but now we don't backpropagate
    
    correct_guesses = 0
    
    for i, batch in enumerate(test_iter):
    #     sentences = batch.context
        senses    = batch.sense

        with torch.no_grad(): # dont collect gradients when testing
            output = model(batch)

        batch_loss = loss(output.view(-1,num_senses), senses.view(-1))
        test_loss += batch_loss.item()

        # finding accuracy
        correct_guesses += torch.sum(torch.eq(torch.argmax(output, dim=1), senses.view(-1)).long())
    
    accuracy = int(correct_guesses) / ((i+1) * batch_size)

    print('>', np.round(test_loss/(i+1), 4))
    print('accuracy: ', accuracy)

In [61]:
# In case of a RuntimeError: The NVIDIA driver on your system is too old (found version 10010),
# run the following command: pip install torch==1.3.1+cu100 torchvision==0.4.2+cu100 -f https://download.pytorch.org/whl/torch_stable.html            

path_to_folder = '.'
train_iter, test_iter, sense, lemma, position, context = dataloader(path_to_folder)

num_words  = len(context.vocab)
num_senses = len(sense.vocab)

model1 = WSDModel_approach1(num_words, num_senses, 50, 50).to(device) 
model2 = WSDModel_approach2(num_words, num_senses, 50, 50).to(device) 

# train_and_test(model1, train_iter, test_iter)

In [62]:
m1 = train(model1, train_iter)

0 60840 1.32412.08991.7512 1.3407
1 60840 0.868319776 0.88150.87670.87590.872153428 0.8696
2 60840 0.71080.7050.7065


In [63]:
m2 = train(model2, train_iter)

0 60840 4.8295.12864.959423832 4.95894.8797
1 60840 4.0989.48194.42934.3796
2 30420 2.7269


In [64]:
test_model(m1, test_iter)

> 0.9117
accuracy:  0.6888640546936629


In [65]:
test_model(m2, test_iter)

> 2.2046
accuracy:  0.41171443597160134


# 2.2 Running a transformer for WSD

In this section of the lab you'll try out the transformer, specifically the BERT model. For this we'll use the huggingface library (https://huggingface.co/).

You can find the documentation for the BERT model here (https://huggingface.co/transformers/model_doc/bert.html) and a general usage guide here (https://huggingface.co/transformers/quickstart.html).

What we're going to do is *fine-tune* the BERT model, i.e. update the weights of a pre-trained model. That is, we have a model that is trained on language modeling, but now we apply it to word sense disambiguation with the word representations it learnt from language modeling.

We'll use the same data splits for training and testing as before, but this time you'll not use a torchtext dataloader. Rather now you create an iterator that collects N sentences (where N is the batch size) then use the **BertTokenizer to transform the sentence into integers**. For your dataloader, remember to:
* Shuffle the data in each batch
* Make sure you get a new iterator for each *epoch*
* Create a vocabulary of *sense-labels* so you can calculate accuracy 

We then pass this batch into the BERT model and train as before. The BERT model will encode the sentence, then we send this encoded sentence into a prediction layer (you can either the the sentence-representation from bert, or the ambiguous word) like before and collect sense predictions.

About the hyperparameters and training:
* For BERT, usually a lower learning rate works best, between 0.0001-0.000001.
* BERT takes alot of resources, running it on CPU will take ages, utilize the GPUs :)
* Since BERT takes alot of resources, use a small batch size (4-8)
* Computing the BERT representation, make sure you pass the mask

**[10 marks]**

In [28]:
learning_rate = 0.000001 # ┗( T﹏T )┛

In [4]:
import random
from transformers import BertTokenizer


def yield_batches(lst, batch_size):
    # Shuffle the data
    random.shuffle(lst)
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]

def dataloader_for_bert(path_to_file, batch_size):
    with open(path_to_file) as file:
        lines = file.readlines()
        #contexts = [ line.split('\t')[3] for line in lines]
        contexts_and_labels = [ (line.split('\t')[3], line.split('\t')[0]) for line in lines]
        #print(contexts_and_labels)
        #Create a vocabulary of sense-labels so you can calculate accuracy
        sense_labels = {line.split('\t')[0] for line in lines}
        # create batches
        iterator = yield_batches(contexts_and_labels, batch_size) # instead of context use context + label
        # use BertTokenizer to encode sentences 
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        input_ids = []
        attention_masks = []
        context_labels = []
        # read batch_size sentences at a time
        for i, batch in enumerate(iterator):
            contexts, labels = zip(*batch)
            tokenized_text = tokenizer.batch_encode_plus(contexts, 
                                                        max_length=128,
                                                        add_special_tokens = True, # add CLS and SEP tokens
                                                        pad_to_max_length=True,
                                                        padding = 'longest',
                                                        truncation=True,
                                                        return_attention_mask=True)
            input_ids.append(tokenized_text['input_ids'])
            attention_masks.append(tokenized_text['attention_mask'])
            # for every batch of tokenized_text we also need to add its labels
            context_labels.append(labels)
       
    return sense_labels, input_ids, attention_masks, context_labels, i #torch.LongTensor(input_ids), torch.FloatTensor(attention_masks)

In [12]:
train_dataloader = dataloader_for_bert("train.csv", batch_size) # (∪.∪ )...zzz

In [5]:
test_dataloader = dataloader_for_bert("test.csv", batch_size)

In [55]:
from transformers import BertModel

class BERT_WSD(nn.Module):
    def __init__(self, sense_labels_size):
        super(BERT_WSD, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(self.bert.config.hidden_size, sense_labels_size)
    
    def forward(self, batch):
        input_ids = torch.LongTensor(batch[0]).to(device)
        attention_mask = torch.FloatTensor(batch[1]).to(device)
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=False) 
        # select cls tokens -> the first element of last_hidden_state 
        predictions = self.classifier(output[0][:,0,:])
        #predictions = self.classifier(output[1]) # automatically selects cls tokens
        return predictions

Writing the Train function
Now we are all set to train our model. This train function is just like how we process a normal Pytorch model. We first set the mode to training, then we iterate through each batch and transfer it to the GPU. Then we pass the input_ids, attention_mask and input_ids to the model. It gives us the output, which consists of loss, logits, hidden_states_output and attention_mask_output. The loss contains the classification loss value. We call the backward function of the loss to calculate the gradients of the parameters of the BERT model. We then call clip_grad_norm_ to prevent the gradients from getting too high or too low. Then we call the optimizer.step() to update the gradients which are calculated by loss.backward(). scheduler.step() is used to update the learning rate according to the scheduler.

In [65]:
import transformers

#sense_labels, input_ids, attention_masks, context_labels, num_batches = train_dataloader
model = BERT_WSD(len(sense_labels)).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = transformers.get_linear_schedule_with_warmup(optimizer,
                                                         num_warmup_steps = 3,#int((num_batches * epochs) * 0.05),
                                                         num_training_steps = num_batches * epochs)

# try warmup steps with 2%, epoch 1 and evaluate

# convert sense_labels to integers
sense_labels_dict = {label: i for i, label in enumerate(sense_labels)}

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [66]:
# (╯°□°）╯︵ ┻━┻ *flips table*
model.train()
#maybe useful ref: https://github.com/kiamehr74/CG20WSD-bert-baseline/blob/33534edabaa9d96105c4e6e109be9bca0a593b80/run.py#L47
for e in range(epochs):
    total_loss = 0
    # NOTE to make stuff faster we could get batches here instead
    for i, batch_tuple in enumerate(zip(input_ids, attention_masks, context_labels)):
        if i == 100:
            assert False
        # output from model
        out = model(batch_tuple)
        #print(torch.argmax(out, dim=1))
        
        target_labels = torch.tensor([sense_labels_dict[label] for label in batch_tuple[2]]).to(device)
        loss  = loss_function(out, target_labels)
        total_loss += loss.item()
        # print average loss for the epoch
        print(total_loss/(i+1), end='\r')

        # backpropagation
        loss.backward()
        
        # optimizing
        optimizer.step()
            
        # clear gradients
        optimizer.zero_grad()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        scheduler.step()
        
    print()
torch.save(model, 'bert_model.pt')

AssertionError: 

In [14]:
model = torch.load("/scratch/bert_model.pt")

In [67]:
sense_labels, input_ids, attention_masks, context_labels, _ = test_dataloader

model.eval()

test_loss = 0
correct_guesses = 0

for i, batch_tuple in enumerate(zip(input_ids, attention_masks, context_labels)):
    #print(batch_tuple)
    with torch.no_grad():
        output = model(batch_tuple)
#     print(output.size())
    target_labels = torch.tensor([sense_labels_dict[label] for label in batch_tuple[2]]).to(device)
#     print(target_labels)
    loss  = loss_function(output, target_labels)
    test_loss += loss.item()
    
    #print("output", torch.argmax(output, dim=1))
    #print("gold", target_labels.view(-1))
    
    correct_guesses += torch.sum(torch.eq(torch.argmax(output, dim=1), target_labels.view(-1)).long())

accuracy = int(correct_guesses) / ((i+1) * batch_size)

print('>', np.round(test_loss/(i+1), 4))
print('BERT accuracy: ', accuracy)

output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([101,  43,  91,  89], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([206, 125,   0, 163], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([200,  76, 166, 133], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([131, 190,  14,  39], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 96, 100, 172, 125], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([101,  89, 163, 216], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 70, 121,  23, 139], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([138,  97,  81,  32], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([211,  40,  50, 111], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([134,  67,  97,  98], device='cuda:2')


output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([131, 104,  70, 140], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([190,   6,  70, 213], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 93, 208,  68, 116], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 81, 101,  46,  98], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 29, 159, 207,  40], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([  7,  87, 148,  94], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 44, 131,   7, 131], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([198, 188, 161, 195], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([141,  71, 213, 188], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 44, 148, 181,  32], device='cuda:2')


output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([155, 101, 121, 125], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([131,  88,  98, 108], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 13,  85,  90, 176], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 20, 182, 208,  52], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 94, 160, 141, 184], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([182,  88,  85, 188], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([161, 220,  32,  29], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 32, 102, 120, 207], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([114,  32, 175, 183], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([211,  32, 100,  94], device='cuda:2')


output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([149,  87,  32,  87], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 93, 170,  72,  91], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([206,  69, 179, 214], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([36, 44, 15, 90], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 30,   8, 189, 176], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([148, 221,  31, 122], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 99,  75, 121,  89], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 97,  80, 143, 101], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([170,  32, 221,  99], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 76, 118,  85,  32], device='cuda:2')
outp

output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([182,  14, 182, 101], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([101, 179, 207,  91], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 94, 164,  44,  60], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([112,  32, 190, 162], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([141, 199, 101,  32], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([  1, 200,  99, 129], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 62, 118, 135,  32], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 32, 145, 120, 138], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 37,  85,  12, 151], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 15,  59, 101,  19], device='cuda:2')


...

output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([101,  75,  80, 197], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 62, 208, 156,  19], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([  8,  60,  58, 133], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([119,  91,   0, 174], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([218,  12,  30,  89], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([133, 127,  81,  44], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([190, 186, 161, 142], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([68, 89, 46, 50], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([148, 101,  84,  32], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([141,  76,  54, 145], device='cuda:2')
outp

output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([211, 214,  70, 176], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([101,  29, 210,  98], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 76,  32,  33, 157], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 43,  85, 112,   1], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 41,  42, 112,  11], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 44, 101,  13,  18], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([115,  87,  32,  44], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([185,  55,  89, 160], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([101,  29, 171,  87], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 52, 127,  55,  40], device='cuda:2')


output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 89, 148, 147, 182], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 76, 101,  32,  85], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([173, 150, 124, 208], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([107,  52, 183, 141], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([27, 44,  7, 37], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 98, 126, 103,  33], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 30,  91, 200, 208], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([203,  32, 208, 216], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 78,  32,  80, 104], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([179, 183, 162,  52], device='cuda:2')
outp

output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([185,  54,  19,  62], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([46, 97, 32, 89], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([141, 144,  44, 100], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([104,   1, 146, 157], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([101,  24,  76, 178], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 70, 111, 101, 141], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([32,  1, 41, 34], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([163, 213, 130, 101], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([157,  32,  12,  14], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([157, 103,  14, 173], device='cuda:2')
output t

output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([144, 175, 211, 178], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 32, 118,  18,  85], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([ 98, 101, 103, 190], device='cuda:2')
output tensor([32, 32, 32, 32], device='cuda:2')
gold tensor([111, 221, 134, 181], device='cuda:2')
output tensor([32, 32], device='cuda:2')
gold tensor([107, 149], device='cuda:2')
> 5.2023
BERT accuracy:  0.0649487246910334


# 3. Evaluation

Explain the difference between the first and second approach. What kind of representations are the different approaches using to predict word-senses? **[4 marks]**

    In the first approach we predicted based on the word selected by the index and in the second approach we predicted based on the final hidden state. The final hidden states contain the memory of the weights of the different layers while the indexed method selects the the hidden state of the word in question.

    The first approach seems to work better based on the lower test loss. We expected the second approach to work better because it contains more information but maybe the first approach is better after all because it is more specific.

Evaluate your model with per-word-form *accuracy* and comment on the results you get, how does the model perform in comparison to the baseline, and how do the models compare to each other? 

Expand on the evaluation by sorting the word-forms by the number of senses they have. Are word-forms with fewer senses easier to predict? Give a short explanation of the results you get based on the number of senses per word.

**[6 marks]**

In [12]:
# baseline accuracy
with open('test.csv') as test:
    t = test.readlines()
    accuracy = 0
    for line in t:
        word, sense = line.split('\t')[1], line.split('\t')[0]
        if sense == baseline[word]:
            accuracy += 1
    print('accuracy of baseline: ', accuracy/len(t))



accuracy of baseline:  0.314266929651545


How does the LSTMs perform in comparison to BERT? What's the difference between representations obtained by the LSTMs and BERT? **[2 marks]**

Representation of LSTM: probability for each of the labels.
Representation of BERT: idk.

What could we do to improve our LSTM word sense disambiguation models and our BERT model? **[4 marks]**

    When it comes to Neural Networks, the data that we use to train, develop and test the model plays a very important role in its performance. As a consequence, we believe that improving the quality and size of data is the most straightforward way to improve the models. For that, we suggest adding more words senses to the dataset. Perhaps having human annotators revising the data could be helpful to check if the sentences chosen are too similar between them or to attest to the quality of the data.

    According to Hadiwinoto et al.(2019) another way to improve LSTM word sense disambiguation models would be "using pre-trained contextualized word representation" i.e., BERT.
    
    To improve BERT, perhaps we could try different forms of finetuning to see what works best. Since our knowledge of BERT is quite limited, we would have to do trial and error to see which approach works best. We could try the approach suggested by Hadiwinoto et al.(2019), to use "linear projection of the hidden vectors, coupled with gating to filter the values".

    Hadiwinoto, C., Ng, H. T., & Gan, W. C. (2019). Improved word sense disambiguation using pre-trained contextualized word representations. arXiv preprint arXiv:1910.00194. https://www.aclweb.org/anthology/D19-1533.pdf

# Readings:

[1] Kågebäck, M., & Salomonsson, H. (2016). Word Sense Disambiguation using a Bidirectional LSTM. arXiv preprint arXiv:1606.03568.

[2] https://cl.lingfil.uu.se/~nivre/master/NLP-LexSem.pdf